In [ ]:
!pip install opencv-python
!pip install opencv-contrib-python

In [1]:
import cv2
import torch
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))

model.to(device)
#print(model)  # display architecture
# Train the model
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

#model.train(data="brain-tumor.yaml", epochs=100, imgsz=640)
#model.save("brain_tumor_yolo.pt")
#localization_loss = torch.nn.MSELoss()

#yolo_metrics=training_loop(model, optimizer, localization_loss, train_loader, val_loader, num_epochs=3)


In [10]:
results = model.train(data="/Users/anton_heden/Documents/Programing/deep-machine-learning/Project/tumor-segmentation/brain_data.yaml", epochs=3, imgsz=64,max_det=1)


New https://pypi.org/project/ultralytics/8.3.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.18 🚀 Python-3.11.0 torch-2.3.1 MPS (Apple M1)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/brain_data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=mps:0, workers=0, project=None, name=train254, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=1, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_co

RuntimeError: Dataset '/brain_data.yaml' error ❌ '/brain_data.yaml' does not exist

In [8]:
model.predict("train/15_jpg.rf.7ded52000bff5f59cb2f6398606ff66a.jpg",save=False,imgsz=640,device=device,max_det=1,show=True)


image 1/1 /Users/anton_heden/Documents/Programing/deep-machine-learning/Project/tumor-segmentation/train/15_jpg.rf.7ded52000bff5f59cb2f6398606ff66a.jpg: 640x640 1 tumor, 139.3ms
Speed: 20.5ms preprocess, 139.3ms inference, 18.6ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'tumor'}
 obb: None
 orig_img: array([[[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3],
         ...,
         [3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]],
 
        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3],
         ...,
         [3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]],
 
        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3],
         ...,
         [3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]],
 
        ...,
 
        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3],
         ...,
         [3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]],
 
        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3],
         ...,
         [3, 3, 3],
         [3, 3, 3],
         [3, 3, 3]],
 
        [[3, 3, 3],
         [3, 3, 3],
         [3, 3, 3],
         ...,
         [3, 3, 3],
         [3, 3, 3],
    